In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import helper_400

%load_ext autoreload
%autoreload 2
helper_400.set_sns_style()

<Figure size 800x600 with 0 Axes>

In [49]:
df = pd.read_csv("data/California.csv", usecols= [
    "STCNTY",
    "COUNTY",
    "FIPS",
    "LOCATION",
    "AREA_SQMI",
    "E_TOTPOP",
    "E_HU",
    "E_HH",
    "EP_POV150",
    "EP_UNEMP",
    "EP_HBURD",
    "EP_NOHSDP",
    "EP_UNINSUR",
    "EP_AGE65",
    "EP_AGE17",
    "EP_DISABL",
    "EP_SNGPNT",
    "EP_LIMENG",
    "EP_MINRTY",
    "EP_NOVEH",
    # "RPL_THEME1",
    # "RPL_THEME2",
    # "RPL_THEME3",
    # "RPL_THEME4",
    "RPL_THEMES",
], dtype={"FIPS":str})

In [50]:
df["FIPSI"]=df["FIPS"].astype(int)

In [51]:
exposure = helper_400.get_exposure_df()

In [52]:
len(exposure)

8673

In [53]:
df = df.merge(exposure, left_on="FIPSI", right_on="GEOID")

In [54]:
avg = pd.DataFrame(np.empty((1, df.shape[1])), columns=df.columns)
# For numerical columns, use mean
for col in df.select_dtypes(include=[np.number]):
    avg.at[0, col] = df[col].mean()

# For non-numerical columns, use mode
for col in df.select_dtypes(exclude=[np.number]):
    avg.at[0, col] = "Aggregated Average"

In [55]:
avg

,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,E_HU,E_HH,EP_POV150,EP_UNEMP,...,heatday,smoke_pm_non_zero,smoke_pm_gt_five,hw,hs,hws,ws,hs5,hws5,ws5
0,6054.952957,Aggregated Average,Aggregated Average,Aggregated Average,17.695392,4361.699758,1559.77528,1440.808601,21.168558,1.206757,...,291.36066,382.35893,134.244206,0.202122,56.476075,0.079903,0.648565,20.980975,0.042315,0.316615


In [56]:
# locations with over three days exposure of the HWS5

df = df[df["hws5"] > 5]

In [57]:
len(df)

16

In [64]:
type(avg)

pandas.core.frame.DataFrame

In [67]:
# Append the mean row to the DataFrame
df = pd.concat([df, avg], ignore_index=True)

In [68]:
df.head()

,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,E_HU,E_HH,EP_POV150,EP_UNEMP,...,heatday,smoke_pm_non_zero,smoke_pm_gt_five,hw,hs,hws,ws,hs5,hws5,ws5
0,6005.0,Amador,06005000402,"Census Tract 4.02, Amador County, California",28.283515,4358.0,2044.0,1671.0,22.1,6.3,...,290.0,672.0,248.0,21.0,121.0,17.0,62.0,48.0,14.0,45.0
1,6009.0,Calaveras,06009000302,"Census Tract 3.02, Calaveras County, California",102.309048,3131.0,1250.0,948.0,31.3,1.0,...,269.0,709.0,259.0,10.0,109.0,9.0,52.0,44.0,7.0,40.0
2,6009.0,Calaveras,06009000505,"Census Tract 5.05, Calaveras County, California",3.157461,993.0,1734.0,486.0,9.2,9.7,...,274.0,637.0,286.0,13.0,104.0,12.0,63.0,53.0,9.0,49.0
3,6017.0,El Dorado,06017030601,"Census Tract 306.01, El Dorado County, California",63.583718,5020.0,2236.0,2045.0,8.7,5.3,...,308.0,724.0,241.0,27.0,134.0,23.0,116.0,45.0,12.0,66.0
4,6023.0,Humboldt,06023010901,"Census Tract 109.01, Humboldt County, California",2.749053,4635.0,1962.0,1816.0,33.4,6.3,...,274.0,522.0,169.0,31.0,87.0,23.0,105.0,30.0,11.0,50.0


In [69]:
df.columns

Index(['STCNTY', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI', 'E_TOTPOP', 'E_HU',
       'E_HH', 'EP_POV150', 'EP_UNEMP', 'EP_HBURD', 'EP_NOHSDP', 'EP_UNINSUR',
       'EP_AGE65', 'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 'EP_LIMENG',
       'EP_MINRTY', 'EP_NOVEH', 'RPL_THEMES', 'FIPSI', 'GEOID', 'wfday',
       'heatday', 'smoke_pm_non_zero', 'smoke_pm_gt_five', 'hw', 'hs', 'hws',
       'ws', 'hs5', 'hws5', 'ws5'],
      dtype='object')

In [70]:
df = df.drop(columns=['FIPSI', 'GEOID'])

In [71]:
df.to_csv("most_exposed_locs_wfday_heat_smoke_305.csv", index=False)